In [ ]:
import torch
!python --version
print(torch.__version__)
print(f"CUDA: {torch.cuda.is_available()}")
# GPUの情報を確認
!nvidia-smi

Python 3.11.12
2.7.1+cu128
CUDA: True
Tue Jul  8 13:40:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.64.03              Driver Version: 575.64.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:07:00.0 Off |                  N/A |
|  0%   51C    P8             16W /  350W |     175MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------

In [ ]:
# google colabなどの環境では、新しいpytorchが必要になります。これらをinstallしてください
#!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
#!pip install -U transformers accelerate bitsandbytes

# 普通のjupyter notebookで動作させる場合は、HuggingFaceのTokenを設定してください
# HF_TOKEN = 'hogehoge'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


In [ ]:

# ローカルLLMの例　（HuggingFaceのアカウントが必要です）

# --- モデルとトークナイザのロード (4-bit量子化設定) ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# モデル名
model_name = "google/gemma-2-2b-jpn-it"
# 4-bit量子化のための設定を作成
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # 量子化のタイプ (nf4が推奨されることが多い)
    bnb_4bit_compute_dtype=torch.bfloat16, # 計算時のデータ型 (bfloat16が効率的)
    bnb_4bit_use_double_quant=True, # 二重量子化を有効にしてメモリをさらに節約
)

print("\nモデルとトークナイザのロードを開始します (4-bit量子化)...")
# トークナイザのロード
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          # token=HF_TOKEN
                                          )
# モデルのロード(RTX5090で300W)
# quantization_config パラメータに上記の設定を渡すことで、モデルが4-bitでロードされます。
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto", # モデルを自動でGPUに配置
    # token=HF_TOKEN
)

print("\nモデルとトークナイザのロードが完了しました。")



モデルとトークナイザのロードを開始します (4-bit量子化)...


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


モデルとトークナイザのロードが完了しました。


In [ ]:
%%time
# テキスト要約のサンプル
# google-colab A100(BF16:312TFLOPS): CPU times: user 73s,    sys: 752 ms, total: 73s Wall time: 1min 13s
#     local RTX5090(BF16:419TFLOPS): CPU times: user 26.1 s, sys: 4.83 s, total: 30.9 sWall time: 29.7 s
#     local RTX3080(BF16:119TFLOPS): CPU times: user 41.5 s, sys: 970 ms, total: 42.4 s Wall time: 42.4 s
#        高火力 H100(BF16:989TFLOPS): CPU times: user 45.3 s, sys: 148 ms, total: 45.4 s Wall time: 45.4 s
#
# テキスト生成パイプラインの準備 ---
# Hugging Faceのpipelineを使うと簡単にテキスト生成ができます。
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("\nテキスト生成パイプラインの準備が完了しました。")

long_text = """
深層学習（ディープラーニング）とは何か？

深層学習（Deep Learning）は、機械学習（Machine Learning）の一種であり、人工知能（AI）の分野において、
現在最も広く活用され、注目を浴びている手法です。
機械学習は、「データから規則性やパターンを学習し、新しいデータに対して適切な予測や判断を行う」ための技術を指します。
その中でも、深層学習は、特に人間の脳神経細胞（ニューロン）の働きを数理モデル化した「ニューラルネットワーク」を、多層化したものです。
ニューラルネットワークの層を深く（deep）積み重ねることで、データに含まれる複雑なパターンや特徴を効果的に抽出・学習できることから、この名前がついています。

深層学習の仕組み（ニューラルネットワークの概要）

深層学習の根幹となるニューラルネットワークは、人工的に作られた神経細胞（ニューロン）の層が複数積み重なった構造をしています。
基本構成は以下の通りです。
入力層 (Input Layer)
データをニューラルネットワークに取り込む最初の層。画像、音声、テキストなどの生データがここに入ります。
中間層（隠れ層：Hidden Layers）
入力データから複雑な特徴を抽出し、それを学習する層。深層学習では、この中間層が複数存在することで、「より複雑な表現」を学習できます。
出力層 (Output Layer)
最終的な結果（分類、回帰予測、生成された画像など）を出力する層。
学習時、各ニューロンは重み（weights）とバイアス（biases）というパラメータを持ちます。
データが入力されると、このパラメータを使って各ニューロンが計算を行い、活性化関数（activation function）を通して出力を生成します。
この出力が次の層に伝播（forward propagation）されます。最終的にネットワークが出力した結果と正解データを比較し、誤差を逆向きに伝播
（back propagation）させてパラメータを調整します。これを「誤差逆伝播法（Back Propagation）」と呼びます。

深層学習が注目される理由

従来の機械学習では、専門家が特徴量を手作業で抽出（特徴エンジニアリング）する必要がありました。
例えば、画像認識なら「色や形状を特徴として選ぶ」などです。
しかし、深層学習は、データから特徴量を自動で抽出する能力を持ち、手作業の特徴設計が不要になります。
これを「表現学習（Representation Learning）」と言います。

深層学習が発展した背景には、以下の3つの要素が大きく寄与しています。
計算資源の向上（GPU、TPUなどの高速な並列処理ハードウェア）
大量かつ高品質なデータセットの整備（画像ならImageNet、テキストならWikipediaなど）
アルゴリズムの改善（Dropout、Batch Normalization、Transformerなど新たなアーキテクチャや技術の開発）
これらが揃ったことで、深層学習は2010年代以降に急速に発展を遂げました。
"""

# --- ステップ5: テキスト生成の実行 ---
# gemma-2-itモデルは、特定の形式のプロンプトで性能を発揮します。
prompt_template = "<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"

length_tbl = [50,100,200,300,500,1000]

for length in length_tbl:
  instruction = f"以下の文を小学5年生でもわかるように、{length}文字程度に要約して文章にしなさい。箇条書きは禁止です。"
  prompt = prompt_template.format(instruction=instruction + f"\n\n{long_text}")

  print("\n--- instruction ---")
  print(instruction)

  # テキスト生成の実行 RTX5090で330W
  outputs = pipe(
      prompt,
      max_new_tokens=1024,
      do_sample=True,
      temperature=0.7,
      top_p=0.95,
  )

  print("\n--- モデルの応答 ---")
  # pipelineの出力はリスト形式で、生成されたテキストは 'generated_text' キーに格納されています
  generated_text = outputs[0]['generated_text']

  # プロンプト部分を除去して、モデルの応答だけを表示
  model_response = generated_text[len(prompt):].strip()
  print(model_response)
  print(f"{len(model_response)} 文字")

print("\n--- 処理完了 ---")



Device set to use cuda:0



テキスト生成パイプラインの準備が完了しました。

--- instruction ---
以下の文を小学5年生でもわかるように、50文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習は、人工知能（AI）で、人間の脳の仕組みを模した技術です。  
複雑なデータから、自動で特徴を見つけ、新しいデータに予測をすることができます。
75 文字

--- instruction ---
以下の文を小学5年生でもわかるように、100文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習は、人工知能の分野で、人間脳の仕組みを参考に、複雑なデータからパターンを学習する技術です。 
ニューラルネットワークという構造で、たくさんの層があり、各層は複雑な特徴を抽出します。
従来は、人工的に特徴量を選びましたが、深層学習は自動で抽出できるようになりました。
これは、大量のデータと、高速な計算機で学習できるようになったためです。
172 文字

--- instruction ---
以下の文を小学5年生でもわかるように、200文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習とは、人工知能(AI)で、人間脳が複雑なパターンを学習する仕組みを模倣した技術です。
機械学習の一種で、たくさんの層（ニューラルネットワーク）で複雑なデータから特徴を自動で抽出します。
この特徴抽出能力は、従来の機械学習では専門家が手作業で抽出する必要があったのに対し、自動で抽出できる点が注目されています。
深層学習は、計算能力の向上、大量のデータセットの整備、新しい技術の開発によって発展しました。
205 文字

--- instruction ---
以下の文を小学5年生でもわかるように、300文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習とは、人工知能(AI)で、人間の脳の仕組みを参考に、複雑なパターンを自動で学習できる技術です。

深層学習は、たくさんの層を重ねて、入力から複雑な特徴を抽出します。 
層を重ねることで、より複雑な表現を学習できます。

深層学習は、従来の機械学習では専門家が特徴量を手作業で決める必要がありましたが

In [ ]:
# --- GPUメモリの解放 ---
import gc
del model
del tokenizer
del pipe
gc.collect()
torch.cuda.empty_cache()
print("\nGPUメモリを解放しました。")


GPUメモリを解放しました。
